In [9]:
import os, pickle
import pandas as pd
import numpy as np
from tqdm import tqdm

In [16]:
data_folder = r'D:\L2_DATA_T0_ETF\train_data\lable_1_combine_data'
code = [
    '513050',
    '513330',
    '518880',
    '159941',
    '513180'
][:1]
label_name = 'paper'
predict_n = 100

down_freq = 3

out_folder = rf'D:\L2_DATA_T0_ETF\train_data\autogluon_of_label_paper_{code[0]}'
if down_freq > 0:
    out_folder = out_folder + f'_d{down_freq}'
out_folder

'D:\\L2_DATA_T0_ETF\\train_data\\autogluon_of_label_paper_513050_d3'

In [11]:
label_cls_idx = 0
for _predict_n in [3, 5, 10, 15, 30, 60, 100]:
    for label in ['paper', 'paper_pct', 'label_1', 'label_1_pct']:
        if _predict_n == predict_n and label == label_name:
            break
        label_cls_idx+=1
print(f'使用标签{label_name}, idx:', label_cls_idx)

def yfunc(threshold, y):
    if y > threshold:
        return 0
    elif y < -threshold:
        return 1
    else:
        return 2

# 定义一个函数来创建展开的数据
def create_expanded_data(df, x, mean_std, need_expand_cols=None):
    total_cols = list(df)
    total_cols_idx = [i for i in range(len(total_cols))]
    if None is  need_expand_cols:
        need_expand_cols = total_cols
        need_expand_cols_idx = total_cols_idx
    else:
        need_expand_cols_idx = [total_cols.index(i) for i in need_expand_cols]

    keep_cols = [i for i in total_cols if i not in need_expand_cols]
    keep_cols_idx = [total_cols.index(i) for i in keep_cols]

    pass_n = x[0][1] - x[0][0]

    expanded_data = []
    for i in range(len(x)):
        a, b = x[i]
        _mean_std = pd.DataFrame(mean_std[i]).iloc[need_expand_cols_idx].reset_index(drop=True)
        _data = df.iloc[a: b, need_expand_cols_idx]

        # 标准化
        _data -= _mean_std.iloc[:, 0].values
        _data /= (_mean_std.iloc[:, 1] - _mean_std.iloc[:, 0]).values

        window = np.concatenate((_data.values.flatten(), df.iloc[b-1, keep_cols].values))
        # expanded_data.append(window)
        window_float32 = window.astype(np.float32)
        expanded_data.append(window_float32)
    
    # 创建新的 DataFrame
    columns = [i for i in need_expand_cols]
    for i in range(1, pass_n):
        for col in need_expand_cols:
            columns.append(f'{col}_{i}')
    columns += keep_cols

    # expanded_df = pd.DataFrame(expanded_data, columns=columns)
    expanded_df = pd.DataFrame(expanded_data, columns=columns, dtype=np.float32)
    return expanded_df

def prepare_data(ids, mean_std, x, y, raw, codes, down_freq=0):
    need_cols = ['OF买1量',
        'OF卖1量',
        'OF买2量',
        'OF卖2量',
        'OF买3量',
        'OF卖3量',
        'OF买4量',
        'OF卖4量',
        'OF买5量',
        'OF卖5量',
        'OF买6量',
        'OF卖6量',
        'OF买7量',
        'OF卖7量',
        'OF买8量',
        'OF卖8量',
        'OF买9量',
        'OF卖9量',
        'OF买10量',
        'OF卖10量']
    # 展平100行数据
    pass_n = 100

    # 筛选标的
    if not isinstance(codes, list):
        codes = [codes]
    idxs = [i for i in range(len(x)) if ids[i].split('_')[0] in codes]
    if len(idxs) == 0:
        return None
    if down_freq > 0:
        idxs = [i for i in idxs if i % down_freq == 0] 
    ids = [ids[i] for i in idxs]
    mean_std = [mean_std[i] for i in idxs]
    x = [x[i] for i in idxs]
    y = [y[i] for i in idxs]

    # 矫正切片索引
    x = [(max(i[0], i[1]-pass_n), i[1]) for i in x]

    # 标签
    y = [i[label_cls_idx] for i in y]
    # y = [yfunc(0.5, i) for i in y]

    raw = raw.loc[:, need_cols]
    raw = create_expanded_data(raw, x, mean_std)

    raw['label'] = pd.Series(y, dtype=np.float32)
    raw['id'] = ids
    # raw['label'] = pd.Series(y, dtype=np.int8)
    return raw


使用标签paper, idx: 24


In [12]:
train_folder = os.path.join(data_folder, 'train')
val_folder = os.path.join(data_folder, 'val')
test_folder = os.path.join(data_folder, 'test')
train_file_list = [os.path.join(train_folder, i) for i in os.listdir(train_folder)] + [os.path.join(val_folder, i) for i in os.listdir(val_folder)]
print(len(train_file_list))
test_file_list = [os.path.join(test_folder, i) for i in os.listdir(test_folder)]
print(len(test_file_list))

# train_file_list = train_file_list[:3]
# test_file_list = test_file_list[:3]

os.makedirs(out_folder, exist_ok=True)
out_train_file = os.path.join(out_folder, 'train.pkl')
out_test_file = os.path.join(out_folder, 'test.pkl')

195
7


In [13]:
train_data = pd.DataFrame()
for file in tqdm(train_file_list):
    # file = train_file_list[0]
    data = prepare_data(*pickle.load(open(file, 'rb')), code, down_freq=down_freq)
    if None is data:
        continue
    train_data = pd.concat([train_data, data])

print('训练数据集长度:', len(train_data))
# train_data.to_csv(out_train_file, index=False)
pickle.dump(train_data, open(out_train_file, 'wb'))

100%|██████████| 195/195 [30:41<00:00,  9.45s/it]


训练数据集长度: 271869


In [14]:
test_data = pd.DataFrame()
for file in tqdm(test_file_list):
    data = prepare_data(*pickle.load(open(file, 'rb')), code, down_freq=down_freq)
    if None is data:
        continue
    test_data = pd.concat([test_data, data])

print('测试数据集长度:', len(test_data))
# test_data.to_csv(out_test_file, index=False)
pickle.dump(test_data, open(out_test_file, 'wb'))

100%|██████████| 7/7 [01:05<00:00,  9.38s/it]


测试数据集长度: 9976
